In [1]:
from scipy.misc import imread
from preprocess.normalize import preprocess_signature
import tf_signet
from tf_cnn_model import TF_CNNModel
import tensorflow as tf
import numpy as np
import pandas as pd
import sys
import os
import scipy.io
from find_largest_image import find_largest
import tqdm 
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from xgboost import XGBClassifier
# from lightgbm import LGBMClassifier
from thundersvm import SVC

c:\users\mert\anaconda3\envs\ml_env\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [2]:
os.listdir(os.path.join(os.getcwd(),'data','GPDSSyntheticSignatures4k'))

['data_features.csv', 'firmasSINTESISmanuscritas', 'visual_features.csv']

In [3]:
data_f = pd.read_csv(os.path.join(os.getcwd(),'data','GPDSSyntheticSignatures4k','data_features.csv'))
visual_f = pd.read_csv(os.path.join(os.getcwd(),'data','GPDSSyntheticSignatures4k','visual_features.csv'))

In [4]:
data_f.columns = ['path','user_id','sig_id','fakeness']
data_f.head()

,path,user_id,sig_id,fakeness
0,/home/mert/Desktop/sigver_dataset/GPDSSyntheti...,3131,29,True
1,/home/mert/Desktop/sigver_dataset/GPDSSyntheti...,3131,7,False
2,/home/mert/Desktop/sigver_dataset/GPDSSyntheti...,3131,4,False
3,/home/mert/Desktop/sigver_dataset/GPDSSyntheti...,3131,26,True
4,/home/mert/Desktop/sigver_dataset/GPDSSyntheti...,3131,17,False


In [5]:
visual_f.head()

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0.0,0.0,2.086571,1.046970,2.012711,0.0,1.471289,0.0,0.0,1.308222,...,0.000000,2.560242,0.000000,0.000000,1.761090,2.054384,0.0,0.0,1.875244,1.555655
1,0.0,0.0,1.909776,1.295273,0.983222,0.0,0.708483,0.0,0.0,0.912948,...,0.000000,2.821531,0.000000,1.145785,2.461549,1.759992,0.0,0.0,2.124749,1.689826
2,0.0,0.0,1.989003,1.175972,1.191586,0.0,1.047355,0.0,0.0,1.942684,...,0.000000,2.109819,0.017639,0.462309,2.615123,1.609518,0.0,0.0,2.022851,1.887849
3,0.0,0.0,2.345391,1.077277,2.091065,0.0,1.449416,0.0,0.0,1.610568,...,0.000000,2.581949,0.000000,0.000000,1.917314,1.687836,0.0,0.0,2.093908,1.857081
4,0.0,0.0,0.980762,0.578907,0.712328,0.0,0.727942,0.0,0.0,1.355375,...,0.234534,2.123285,0.000000,1.186418,2.821104,1.428768,0.0,0.0,1.720274,1.232280


In [6]:
print(data_f.shape)
print(visual_f.shape)
print(len(data_f['user_id'].unique()), 'users are enrolled in the dataset')
user_count = len(data_f['user_id'].unique())

(216000, 4)
(216000, 2048)
4000 users are enrolled in the dataset


In [7]:
user_id_list = np.arange(1,len(data_f['user_id'].unique())/5+1)
train_idx, test_idx = train_test_split(user_id_list, train_size=0.8, random_state=1923)

c:\users\mert\anaconda3\envs\ml_env\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [87]:
# Filter train users and test users.
data_f_train = data_f.loc[data_f['user_id'].isin(train_idx)]
data_f_test = data_f.loc[data_f['user_id'].isin(test_idx)]
# Filter fakes and genuines.
data_f_train_gen = data_f_train.loc[data_f_train['fakeness']==False]
data_f_train_fake = data_f_train.loc[data_f_train['fakeness']==True]
data_f_train_gen_5 = data_f_train_gen.loc[data_f_train_gen['sig_id'].isin(np.arange(1,6))]
data_f_train_gen_14 = data_f_train_gen.loc[data_f_train_gen['sig_id'].isin(np.arange(1,15))]
data_f_train_gen_valid_10 = data_f_train_gen.loc[data_f_train_gen['sig_id'].isin(np.arange(15,25))]

In [91]:
y_valid_fakes = data_f_train_fake.loc[data_f_train_fake['user_id']==3]
y_valid_gens = data_f_train_gen_valid_10.loc[data_f_train_gen_valid_10['user_id']==3]

In [98]:
visual_f.iloc[y_valid_gens.index]

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
36673,0.0,0.0,1.471385,2.274128,1.222940,0.0,0.562180,0.0,0.000000,1.165962,...,0.531699,1.641033,0.000000,0.894280,1.619027,1.160282,0.00000,0.00000,1.707765,2.314837
36683,0.0,0.0,1.787887,1.812570,0.992943,0.0,1.176085,0.0,0.000000,1.392810,...,0.399828,1.262930,0.000000,1.153502,1.887194,0.605296,0.00000,0.00000,1.816247,1.872828
36687,0.0,0.0,0.749076,1.861110,1.061019,0.0,0.277058,0.0,0.000000,1.650121,...,0.582539,2.188168,0.000000,1.531710,2.110659,0.729954,0.00000,0.00000,1.790474,2.200254
36692,0.0,0.0,0.584838,2.099784,1.232006,0.0,0.627573,0.0,0.000000,1.218586,...,0.396872,2.200965,0.000000,0.981013,1.895179,0.940632,0.00000,0.00000,1.779428,2.409850
36693,0.0,0.0,0.000000,0.405680,0.969205,0.0,0.000000,0.0,0.811739,0.253086,...,0.945615,0.820946,0.431435,1.297336,1.519696,0.652986,0.50921,0.49326,0.000000,1.095153
36704,0.0,0.0,1.557531,2.188437,1.480708,0.0,0.869263,0.0,0.000000,1.227091,...,0.084415,2.387735,0.000000,0.726073,1.941355,1.292338,0.00000,0.00000,1.992474,2.354172
36706,0.0,0.0,1.720532,1.411796,1.976025,0.0,1.570951,0.0,0.000000,1.279853,...,0.000000,2.152899,0.000000,0.000000,1.754323,1.558308,0.00000,0.00000,1.376908,2.054243
36708,0.0,0.0,1.453852,1.167280,2.111248,0.0,1.511664,0.0,0.000000,1.194573,...,0.000000,2.354470,0.000000,0.000000,1.625385,1.612748,0.00000,0.00000,1.399778,1.971535
36716,0.0,0.0,1.575244,2.371814,1.053659,0.0,0.338625,0.0,0.000000,1.312291,...,0.000000,1.695810,0.000000,0.975037,1.744182,1.093950,0.00000,0.00000,1.140224,2.091554
36718,0.0,0.0,0.845514,2.090253,1.250455,0.0,0.554532,0.0,0.000000,1.380262,...,0.435188,2.140426,0.000000,1.361629,2.085245,0.793011,0.00000,0.00000,1.881504,2.071706


In [103]:
data_f_train_gen_14['user_id']==user_id

60         True
62         True
65         True
72         True
74         True
75         True
77         True
78         True
83         True
84         True
89         True
92         True
99         True
100        True
757       False
758       False
760       False
767       False
769       False
776       False
780       False
788       False
789       False
791       False
795       False
797       False
803       False
804       False
811       False
824       False
          ...  
215013    False
215022    False
215622    False
215628    False
215631    False
215632    False
215636    False
215639    False
215641    False
215645    False
215647    False
215650    False
215654    False
215668    False
215670    False
215672    False
215734    False
215735    False
215740    False
215746    False
215747    False
215750    False
215759    False
215765    False
215769    False
215770    False
215771    False
215775    False
215778    False
215780    False
Name: user_id, Length: 8

In [108]:
for user_id in tqdm.tqdm(data_f_train_gen_14['user_id'].unique()[0:5]):
    y_train = data_f_train_gen_14['user_id']==user_id
    X_train = visual_f.iloc[y_train.index]
    # clf = XGBClassifier(tree_method='gpu_hist',verbosity=2,predictor='gpu_predictor', scale_pos_weight=2*8946/14)
    clf = SVC(class_weight='balanced')
    clf.fit(X_train, y_train)
    y_valid_fakes = data_f_train_fake.loc[data_f_train_fake['user_id']==user_id]
    y_valid = len(y_valid_fakes.index)*[False]
    X_valid = visual_f.iloc[y_valid_fakes.index]
    print(clf.score(X_valid,y_valid))
    y_valid_gens = data_f_train_gen_valid_10.loc[data_f_train_gen_valid_10['user_id']==user_id]
    y_valid = len(y_valid_gens.index)*[True]
    X_valid = visual_f.iloc[y_valid_gens.index]
    print(clf.score(X_valid,y_valid))


  0%|                                                                                            | 0/5 [00:00<?, ?it/s]c:\users\mert\anaconda3\envs\ml_env\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.7
0.9



 20%|████████████████▊                                                                   | 1/5 [00:15<01:02, 15.72s/it]c:\users\mert\anaconda3\envs\ml_env\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.7
0.9



 40%|█████████████████████████████████▌                                                  | 2/5 [00:33<00:49, 16.48s/it]c:\users\mert\anaconda3\envs\ml_env\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


...


 60%|██████████████████████████████████████████████████▍                                 | 3/5 [00:47<00:31, 15.56s/it]c:\users\mert\anaconda3\envs\ml_env\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.7
0.8



 80%|███████████████████████████████████████████████████████████████████▏                | 4/5 [01:01<00:15, 15.16s/it]c:\users\mert\anaconda3\envs\ml_env\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.4
0.9



100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:27<00:00, 18.49s/it]

In [74]:
data_f_train_gen

,path,user_id,sig_id,fakeness
54,/home/mert/Desktop/sigver_dataset/GPDSSyntheti...,491,21,False
55,/home/mert/Desktop/sigver_dataset/GPDSSyntheti...,491,18,False
57,/home/mert/Desktop/sigver_dataset/GPDSSyntheti...,491,17,False
60,/home/mert/Desktop/sigver_dataset/GPDSSyntheti...,491,11,False
62,/home/mert/Desktop/sigver_dataset/GPDSSyntheti...,491,13,False
63,/home/mert/Desktop/sigver_dataset/GPDSSyntheti...,491,22,False
64,/home/mert/Desktop/sigver_dataset/GPDSSyntheti...,491,23,False
65,/home/mert/Desktop/sigver_dataset/GPDSSyntheti...,491,9,False
68,/home/mert/Desktop/sigver_dataset/GPDSSyntheti...,491,15,False
72,/home/mert/Desktop/sigver_dataset/GPDSSyntheti...,491,8,False


In [80]:
x = clf.predict(visual_f)

In [86]:
x[62]

True

In [13]:
data_f_test_gen = data_f_test.loc[data_f_test['fakeness']==False]
data_f_test_fake = data_f_test.loc[data_f_test['fakeness']==True]
data_f_test_gen_5 = data_f_test_gen.loc[data_f_test_gen['sig_id'].isin(np.arange(1,6))]
data_f_test_gen_14 = data_f_test_gen.loc[data_f_test_gen['sig_id'].isin(np.arange(1,15))]

In [15]:
X_valid = visual_f.iloc[data_f_test_gen_14.index]
y_valid = data_f_test_gen_14['user_id']

In [16]:
len(y_train.unique())

640

In [20]:
y_train.head()

60    491
62    491
65    491
72    491
74    491
Name: user_id, dtype: int64

In [ ]:
clf=XGBClassifier(tree_method='gpu_hist', verbosity=2, predictor='gpu_predictor')
clf.fit(X_train, y_train)
print(clf.score(X_train, y_train))

In [ ]:
'''
clf=LGBMClassifier(device='gpu')
clf.fit(X_train, y_train)
'''

In [ ]:
# TODO Add benchmark metric above. 
# TODO CNN Model & Training, thundersvm model, gbm model, re-implementation of the paper.